In [1]:
import numpy as np
import pandas as pd

In [35]:
#读取合并数据
train_data=pd.read_csv("./data/train.csv")
test_data=pd.read_csv("./data/test.csv")

full_data=train_data.append(test_data, ignore_index=True)

D:\Python_Anaconda\Anaconda\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [36]:
#数据清洗
#full_data.describe()
#full_data.info()

full_data['Age']=full_data['Age'].fillna(full_data['Age'].mean())
full_data['Fare']=full_data['Fare'].fillna(full_data['Fare'].mean())
full_data['Cabin']=full_data['Cabin'].fillna('U')
full_data['Embarked']=full_data['Embarked'].fillna('S')
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
Age            1309 non-null float64
Cabin          1309 non-null object
Embarked       1309 non-null object
Fare           1309 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


In [37]:
#特征提取
sex_mapDict={'male':1, 'female':0}
full_data['Sex']=full_data['Sex'].map(sex_mapDict)

embarkedDf=pd.DataFrame()
embarkedDf=pd.get_dummies(full_data['Embarked'], prefix='Embarked')
full_data=pd.concat([full_data, embarkedDf], axis=1)
full_data.drop('Embarked', axis=1, inplace=True)

pclassDf=pd.DataFrame()
pclassDf=pd.get_dummies(full_data['Pclass'], prefix='Pclass')
full_data=pd.concat([full_data, pclassDf], axis=1)
full_data.drop('Pclass', axis=1, inplace=True)

def getTitle(name):
    str1=name.split(',')[1]
    str2=str1.split('.')[0]
    str3=str2.strip()
    return str3
titleDf=pd.DataFrame()
titleDf['Title']=full_data['Name'].map(getTitle)
title_mapDict={
    'Capt': 'Officer',
    'Col': 'Officer',
    'Major': 'Officer',
    'Jonkheeer': 'Royalty',
    'Don': 'Royalty',
    'Sir': 'Royalty',
    'Dr': 'Officer',
    'Rev': 'Officer',
    'the Countess': 'Royalty',
    'Dona': 'Royalty',
    'Mme': 'Mrs',
    'Mlle': 'Miss',
    'Ms': 'Mrs',
    'Mr': 'Mr',
    'Mrs': 'Mrs',
    'Miss': 'Miss',
    'Master': 'Master',
    'Lady' : 'Royalty'
}
titleDf['Title']=titleDf['Title'].map(title_mapDict)
titleDf=pd.get_dummies(titleDf['Title'])
full_data=pd.concat([full_data, titleDf], axis=1)
full_data.drop('Name', axis=1, inplace=True)

cabinDf=pd.DataFrame()
full_data['Cabin']=full_data['Cabin'].map(lambda c:c[0])
cabinDf=pd.get_dummies(full_data['Cabin'], prefix='Cabin')
full_data=pd.concat([full_data, cabinDf], axis=1)
full_data.drop('Cabin', axis=1, inplace=True)

familyDf=pd.DataFrame()
familyDf['Familysize']=full_data['SibSp']+full_data['Parch']+1
familyDf['Family_Single']=familyDf['Familysize'].map(lambda s:1 if s==1 else 0)
familyDf['Family_Small']=familyDf['Familysize'].map(lambda s:1 if 2<= s <=4 else 0)
familyDf['Family_Large']=familyDf['Familysize'].map(lambda s:1 if 5<= s else 0)
full_data=pd.concat([full_data, familyDf], axis=1)

ageDf=pd.DataFrame()
ageDf['Child']=full_data['Age'].map(lambda a:1 if 0< a <=6 else 0)
ageDf['Teenager']=full_data['Age'].map(lambda a:1 if 6< a <18 else 0)
ageDf['Youth']=full_data['Age'].map(lambda a:1 if 18<= a <=40 else 0)
ageDf['Middle_aged']=full_data['Age'].map(lambda a:1 if 40< a <=60 else 0)
ageDf['Older']=full_data['Age'].map(lambda a:1 if 60< a  else 0)
full_data=pd.concat([full_data, ageDf], axis=1)
full_data.drop('Age', axis=1, inplace=True)

full_data.shape
full_data.head()

,Fare,Parch,PassengerId,Sex,SibSp,Survived,Ticket,Embarked_C,Embarked_Q,Embarked_S,...,Cabin_U,Familysize,Family_Single,Family_Small,Family_Large,Child,Teenager,Youth,Middle_aged,Older
0,7.2500,0,1,1,1,0.0,A/5 21171,0,0,1,...,1,2,0,1,0,0,0,1,0,0
1,71.2833,0,2,0,1,1.0,PC 17599,1,0,0,...,0,2,0,1,0,0,0,1,0,0
2,7.9250,0,3,0,0,1.0,STON/O2. 3101282,0,0,1,...,1,1,1,0,0,0,0,1,0,0
3,53.1000,0,4,0,1,1.0,113803,0,0,1,...,0,2,0,1,0,0,0,1,0,0
4,8.0500,0,5,1,0,0.0,373450,0,0,1,...,1,1,1,0,0,0,0,1,0,0


In [40]:
#特征选择
corrDf=full_data.corr()
corrDf['Survived'].sort_values(ascending=False)
full_data_X=pd.concat([titleDf, pclassDf, familyDf, full_data['Fare'], full_data['Sex'], cabinDf, embarkedDf], axis=1)
full_data_X.head()

,Master,Miss,Mr,Mrs,Officer,Royalty,Pclass_1,Pclass_2,Pclass_3,Familysize,...,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1,0,0,0,0,0,1,2,...,0,0,0,0,0,0,1,0,0,1
1,0,0,0,1,0,0,1,0,0,2,...,1,0,0,0,0,0,0,1,0,0
2,0,1,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,1,0,0,1
3,0,0,0,1,0,0,1,0,0,2,...,1,0,0,0,0,0,0,0,0,1
4,0,0,1,0,0,0,0,0,1,1,...,0,0,0,0,0,0,1,0,0,1


In [69]:
#构建训练模型
source_X=full_data_X.loc[0:train_data.shape[0]-1, :]
source_Y=full_data.loc[0:train_data.shape[0]-1, 'Survived']
pred_X=full_data_X.loc[train_data.shape[0]:, :]

from sklearn.model_selection import train_test_split
train_X, test_X, train_Y, test_Y = train_test_split(source_X, source_Y, train_size=0.8)

from sklearn.linear_model import LogisticRegression
model=LogisticRegression()

model.fit(train_X, train_Y)
model.score(test_X, test_Y)

D:\Python_Anaconda\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8659217877094972

In [79]:
#模型预测
pred_Y=model.predict(pred_X)
pred_Y=pred_Y.astype(int)

passenger_id=test_data.loc[:,'PassengerId']
#passenger_id.head()

predDf=pd.DataFrame({'PassengerId': passenger_id, 'Survived': pred_Y})
predDf.head()


,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [80]:
#预测结果保存
predDf.to_csv('./titanic_pred.csv', index=False)